In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tqdm import(tqdm)

class DTMKL:
    def __init__(self, X_train_A, Y_train_A, X_train_T, Y_train_T,
                  X_unlabeled_T=None,
                 kernel_types=['linear', 'rbf'], 
                 C=1.0,theta = 1, eta=0.1,
                 gamma_rbf=1.0, epsilon=1e-3, max_iter=100):
        """
        Domain Transfer Multiple Kernel Learning (DTMKL) implementation.
        
        Parameters:
        - X_train_A: Features from auxiliary domain
        - Y_train_A: Labels from auxiliary domain
        - X_train_T: Features from target domain (labeled)
        - Y_train_T: Labels from target domain
        - X_unlabeled_T: Unlabeled features from target domain (for MMD calculation)
        - kernel_types: Types of base kernels ['linear', 'rbf', 'poly1.5']
        - C: SVM regularization parameter
        - eta: Trade-off parameter between domain adaptation and classification error
        - gamma_rbf: RBF kernel parameter
        - degree_poly: Polynomial kernel degree
        - epsilon: Convergence threshold
        - max_iter: Maximum number of iterations
        """
        # Initialize data
        self.X_train_A = X_train_A
        self.Y_train_A = Y_train_A.reshape(-1, 1)
        self.X_train_T = X_train_T
        self.Y_train_T = Y_train_T.reshape(-1, 1)
        self.theta = theta
        # Handle unlabeled target data
        if X_unlabeled_T is None:
            # If no unlabeled data is provided, use labeled target data for MMD
            self.X_unlabeled_T = X_train_T
        else:
            self.X_unlabeled_T = X_unlabeled_T
            
        # Combine labeled data from both domains
        self.X_labeled = np.vstack([X_train_A, X_train_T])
        self.Y_labeled = np.vstack([self.Y_train_A, self.Y_train_T]).flatten()
        self.nA = len(X_train_A)
        self.nT_labeled = len(X_train_T)
        self.nT_unlabeled = len(self.X_unlabeled_T)
        
        # All data for MMD calculation
        self.X_all_mmd = np.vstack([X_train_A,X_train_T,self.X_unlabeled_T])
        self.nA_mmd = self.nA
        self.nT_mmd = self.nT_labeled + self.nT_unlabeled
        
        # Kernel parameters
        self.kernel_types = kernel_types
        self.M = len(kernel_types)
        self.gamma_rbf = gamma_rbf
        self.degree_poly = degree_poly
        
        # Optimization parameters
        self.C = C
        self.eta = eta
        self.epsilon = epsilon
        self.max_iter = max_iter
        
        # Initialize domain indicator vector for MMD
        self.s = self._create_s_vector()
        
        # Precompute kernel matrices
        self.Km_list_labeled = self._precompute_base_kernels(self.X_labeled, self.X_labeled)
        self.Km_list_mmd = self._precompute_base_kernels(self.X_all_mmd, self.X_all_mmd)
        
        # Initialize kernel weights
        self.d = np.ones(self.M) / self.M
        self.alpha = None
        self.b = 0
        self.svm = None

    def _create_s_vector(self):
        """
        Create domain indicator vector for MMD calculation.
        Returns a vector where auxiliary domain entries are 1/nA and 
        target domain entries are -1/nT
        """
        s_A = np.full(self.nA_mmd, 1.0/self.nA_mmd)
        s_T = np.full(self.nT_mmd, -1.0/self.nT_mmd)
        return np.concatenate([s_A, s_T]).reshape(-1,1)  # (n_samples, 1)

    def _kernel_function(self, X1, X2, kernel_type):
        """Calculate kernel matrix for a given kernel type"""
        if kernel_type == 'linear':
            return X1 @ X2.T
        elif kernel_type == 'rbf':
            gamma = 1.0 / (X1.shape[1] * self.gamma_rbf)
            pairwise_dists = np.sum(X1**2, axis=1)[:, np.newaxis] + \
                             np.sum(X2**2, axis=1) - 2 * X1 @ X2.T
            return np.exp(-gamma * pairwise_dists)
        elif kernel_type.startswith("poly"):
            degree = float(kernel_type[4:])
            return (X1 @ X2.T + 1) ** degree
        else:
            raise ValueError(f"Unsupported kernel type: {kernel_type}")

    def _precompute_base_kernels(self, X1, X2):
        """Precompute all base kernel matrices"""
        return [self._kernel_function(X1, X2, kt) for kt in self.kernel_types]

    def compute_mmd_vector(self):
        """
        Compute the MMD vector p = [tr(K1S), ..., tr(KmS)]^T,
        where S = ss^T and s is the domain indicator vector
        """
        return np.array([np.trace(Km @ (self.s @ self.s.T)) for Km in self.Km_list_mmd]).reshape(-1, 1)

    def _combine_kernels(self, d, kernel_matrices):
        """Combine multiple kernel matrices using weights d"""
        return np.sum([d[m] * kernel_matrices[m] for m in range(self.M)], axis=0)

    def _compute_gradient(self, d):
        """
        Compute the gradient of the objective function:
        ∇h = pp^T d + η ∇J
        where p is the MMD vector and ∇J is the gradient of SVM objective
        """
        # MMD part gradient: pp^T * d
        p = self.compute_mmd_vector()
        grad_part1 = (p @ p.T) @ d.reshape(-1, 1)
        
        # SVM objective gradient: η * ∂J/∂d
        y = self.Y_labeled.reshape(-1, 1)
        alpha_y = self.alpha.reshape(-1, 1) * y
        grad_J = np.array([-0.5 * (alpha_y.T @ Km @ alpha_y).item() 
                          for Km in self.Km_list_labeled])
        grad_part2 = self.theta * grad_J.reshape(-1, 1)
        
        return grad_part1 + grad_part2

    def _compute_hessian(self, d):
        """
        Compute the Hessian matrix of the objective function
        H = pp^T + η ∇²J
        """
        p = self.compute_mmd_vector()
        hessian = p @ p.T
        
        # Add a small value to diagonal to ensure positive definiteness
        return hessian + 1e-2 * np.eye(self.M)

    def _project_simplex(self, v):
        """
        Project vector v onto the probability simplex:
        d ≥ 0, Σd = 1
        """
        u = np.sort(v)[::-1]
        cumsum = np.cumsum(u)
        rho = np.where(u > (cumsum - 1) / (np.arange(1, len(v)+1)))[0][-1]
        theta = (cumsum[rho] - 1) / (rho+1)
        return np.maximum(v - theta, 0)

    def fit(self):
        """Main training loop for DTMKL"""
        prev_obj = np.inf
        for iter in tqdm(range(self.max_iter)):
            print(self.d)
            # Step 1: Fix d, train SVM
            K_combined = self._combine_kernels(self.d, self.Km_list_labeled)
            self.svm = SVC(kernel='precomputed', C=self.C)
            self.svm.fit(K_combined, self.Y_labeled)
            
            # Reconstruct full alpha array
            self.alpha = np.zeros(len(self.Y_labeled))
            self.alpha[self.svm.support_] = self.svm.dual_coef_[0] * self.Y_labeled[self.svm.support_]
            self.b = self.svm.intercept_[0]
            
            # Step 2: Fix alpha, update d using second-order method
            gradient = self._compute_gradient(self.d)
            hessian = self._compute_hessian(self.d)
            
            # Use Newton's method with line search
            gradient_col = gradient.reshape(-1, 1)  # 确保列向量
            update_direction = -(np.linalg.inv(hessian) @ gradient_col).flatten()
            
            # Line search for step size
            step_size = self.eta
            d_new = self._project_simplex(self.d - step_size * update_direction)
            
            # # Calculate objective function value
            # p = self.compute_mmd_vector()
            # obj_mmd = 0.5 * (self.d.T @ p)**2
            
            # K_combined = self._combine_kernels(self.d, self.Km_list_labeled)
            # obj_svm = 0.5 * self.alpha @ K_combined @ self.alpha - np.sum(self.alpha)
            # obj = obj_mmd + self.eta * obj_svm
            
            # # Print progress
            # if verbose and (iter % 10 == 0 or iter == self.max_iter-1):
            #     acc = accuracy_score(self.Y_labeled, self.svm.predict(K_combined))
            #     print(f"Iter {iter}: Obj={obj.item():.4f}, Acc={acc:.4f}, d={np.round(self.d,3)}")
            
            # # Check convergence
            # if np.linalg.norm(d_new - self.d) < self.epsilon and abs(obj - prev_obj) < self.epsilon:
            #     if verbose: 
            #         print(f"Converged at iteration {iter}")
            #     break
                
            self.d = d_new
            print(self.d)
            #prev_obj = obj

    def predict(self, X_test):
        """Predict labels for new test data"""
        # Compute kernel matrices between test and training data
        K_test_list = []
        for kt in self.kernel_types:
            K_test = self._kernel_function(X_test, self.X_labeled, kt)
            K_test_list.append(K_test)
        
        # Combine kernels
        K_test_combined = self._combine_kernels(self.d, K_test_list)
        
        # Use SVM to predict
        return self.svm.predict(K_test_combined)

    def evaluate(self, X_test, Y_test):
        """Evaluate model performance"""
        y_pred = self.predict(X_test)
        return accuracy_score(Y_test, y_pred)



In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')
import os
from sklearn.model_selection import train_test_split



def load_newsgroup_data(setting='comp_vs_rec'):
    """
    Load and prepare 20 Newsgroups dataset for domain adaptation experiment.
    
    Parameters:
    - setting: Which setting to use ('comp_vs_rec', 'comp_vs_sci', or 'comp_vs_talk')
    
    Returns:
    - Data for auxiliary and target domains
    """
    print(f"Loading {setting} data...")
    
    # Define category mappings according to the paper's Table 1
    settings = {
        'comp_vs_rec': {
            'auxiliary': ['comp.windows.x', 'rec.sport.hockey'],
            'target': ['comp.sys.ibm.pc.hardware', 'rec.motorcycles'],
            'positive_class': 'comp',
            'negative_class': 'rec'
        },
        'comp_vs_sci': {
            'auxiliary': ['comp.windows.x', 'sci.crypt'],
            'target': ['comp.sys.ibm.pc.hardware', 'sci.med'],
            'positive_class': 'comp',
            'negative_class': 'sci'
        },
        'comp_vs_talk': {
            'auxiliary': ['comp.windows.x', 'talk.politics.mideast'],
            'target': ['comp.sys.ibm.pc.hardware', 'talk.politics.guns'],
            'positive_class': 'comp',
            'negative_class': 'talk'
        }
    }
    
    # Get the specific setting
    current_setting = settings[setting]
    
    # Load the auxiliary domain data
    auxiliary_categories = current_setting['auxiliary']
    auxiliary_data = fetch_20newsgroups(subset='all', 
                                       categories=auxiliary_categories,
                                       shuffle=True, 
                                       random_state=42)
    
    # Load the target domain data
    target_categories = current_setting['target']
    target_data = fetch_20newsgroups(subset='all', 
                                    categories=target_categories,
                                    shuffle=True, 
                                    random_state=42)
    
    # Create labels
    # Positive class (comp) = 1, Negative class (rec/sci/talk) = -1
    auxiliary_labels = []
    for target_idx in auxiliary_data.target:
        category = auxiliary_data.target_names[target_idx]
        if category.startswith(current_setting['positive_class']):
            auxiliary_labels.append(1)
        else:
            auxiliary_labels.append(-1)
    auxiliary_labels = np.array(auxiliary_labels)

    target_labels = []
    for target_idx in target_data.target:
        category = target_data.target_names[target_idx]
        if category.startswith(current_setting['positive_class']):
            target_labels.append(1)
        else:
            target_labels.append(-1)
    target_labels = np.array(target_labels)
    

    
    # Create feature vectors - use TF-IDF on the text data
    vectorizer = TfidfVectorizer(min_df=5, max_df=0.9, sublinear_tf=True, use_idf=True)
    
    # Fit the vectorizer on all data
    all_texts = auxiliary_data.data + target_data.data
    vectorizer.fit(all_texts)
    
    # Transform the auxiliary and target data
    auxiliary_features = vectorizer.transform(auxiliary_data.data).toarray()
    target_features = vectorizer.transform(target_data.data).toarray()
    
    print(f"Auxiliary domain: {len(auxiliary_labels)} samples")
    print(f"Target domain: {len(target_labels)} samples")
    
    return auxiliary_features, auxiliary_labels, target_features, target_labels
X_aux, y_aux, X_tar, y_tar = load_newsgroup_data('comp_vs_talk')
print("X_aux",X_aux)
print("y_aux",y_aux)
print("X_tar",X_tar)
print("y_tar",y_tar)

        # 随机选择标记样本
pos_indices = np.where(y_tar == 1)[0]
neg_indices = np.where(y_tar == -1)[0]
        
np.random.seed(123)  # 设置随机种子以确保可重复性
labeled_samples_per_class=5
# 确保选择的样本数不超过可用样本数
n_pos = min(labeled_samples_per_class, len(pos_indices))
n_neg = min(labeled_samples_per_class, len(neg_indices))
        
if n_pos < labeled_samples_per_class or n_neg < labeled_samples_per_class:
    print(f"Warning: Requested {labeled_samples_per_class} samples per class, but only found {n_pos} positive and {n_neg} negative samples")
        
pos_labeled_idx = np.random.choice(pos_indices, n_pos, replace=False)
neg_labeled_idx = np.random.choice(neg_indices, n_neg, replace=False)
        
labeled_idx = np.concatenate([pos_labeled_idx, neg_labeled_idx])
unlabeled_idx = np.array([i for i in range(len(y_tar)) if i not in labeled_idx])
        
# 分割目标域数据为标记和未标记
X_tar_labeled = X_tar[labeled_idx]
y_tar_labeled = y_tar[labeled_idx]
X_tar_unlabeled = X_tar[unlabeled_idx]
y_tar_unlabeled = y_tar[unlabeled_idx]  # 用于评估的真实标签
# 只使用线性核
C=5
kernel_types = ['linear',"poly1.5","poly1.6","poly1.7","poly1.8","poly1.9","poly2.0"]
dtmkl = DTMKL(
      X_train_A=X_aux,
      Y_train_A=y_aux, 
      X_train_T=X_tar_labeled,
      Y_train_T=y_tar_labeled,
      X_unlabeled_T=X_tar_unlabeled,
      kernel_types=kernel_types,  # 只使用线性核
      C=C,
      theta = 2e-3,
      eta=2e-3,
      max_iter=50  # 减少迭代次数
)
                        
                        
# 训练DTMKL模型
dtmkl.fit(verbose=False)
                        
# 预测
y_pred = dtmkl.predict(X_tar_unlabeled)  
print("y_pred",y_pred)
print((len(y_pred)-np.count_nonzero(y_pred-y_tar_unlabeled))/len(y_pred))     
clf = SVC(kernel='linear', C=C)
clf.fit(X_tar_labeled, y_tar_labeled)
y_pred1 = clf.predict(X_tar_unlabeled)
print("ypre-svm_T",y_pred1)
print((len(y_pred1)-np.count_nonzero(y_pred1-y_tar_unlabeled))/len(y_pred1))

clf = SVC(kernel='linear', C=C)
clf.fit(X_aux, y_aux)
y_pred2 = clf.predict(X_tar_unlabeled)
print("ypre-svm_A",y_pred2)
print((len(y_pred2)-np.count_nonzero(y_pred2-y_tar_unlabeled))/len(y_pred2))

X_combined = np.vstack([X_aux, X_tar_labeled])
y_combined = np.concatenate([y_aux, y_tar_labeled])
clf = SVC(kernel='linear', C=C)
clf.fit(X_combined, y_combined)
y_pred3 = clf.predict(X_tar_unlabeled)
print("ypre-svm_AT",y_pred2)
print((len(y_pred3)-np.count_nonzero(y_pred3-y_tar_unlabeled))/len(y_pred3))


 
        


Loading comp_vs_talk data...
Auxiliary domain: 1928 samples
Target domain: 1892 samples
X_aux [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
y_aux [ 1 -1 -1 ...  1  1  1]
X_tar [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
y_tar [-1  1  1 ... -1  1 -1]


  0%|          | 0/50 [00:00<?, ?it/s]

[0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]


  2%|▏         | 1/50 [00:14<12:08, 14.86s/it]

[0.15002231 0.14495309 0.14374406 0.14245893 0.1410923  0.13963838
 0.13809093]
[0.15002231 0.14495309 0.14374406 0.14245893 0.1410923  0.13963838
 0.13809093]


...

 98%|█████████▊| 49/50 [13:11<00:16, 16.18s/it]

[0.61171134 0.22448184 0.1314563  0.03235053 0.         0.
 0.        ]
[0.61171134 0.22448184 0.1314563  0.03235053 0.         0.
 0.        ]


100%|██████████| 50/50 [13:28<00:00, 16.16s/it]

[0.6228549  0.22371085 0.12780636 0.02562789 0.         0.
 0.        ]


y_pred [-1  1  1 ... -1  1 -1]
0.9011689691817216
ypre-svm_T [-1  1  1 ... -1  1 -1]
0.8804463336875664
ypre-svm_A [-1  1  1 ... -1  1 -1]
0.8767268862911796
ypre-svm_AT [-1  1  1 ... -1  1 -1]
0.9017003188097769
